In [1]:
%load_ext cython

In [2]:
import pandas as pd
import numpy as np

### PLS

In [3]:
def PLS(X, Y):
    t_ = 0
    t = 0
    W = {}
    B = {}
    P = {}
    Q = {}
    E = X
    F = Y
    
    y_shape = Y.shape[1]
    rank_x = np.linalg.matrix_rank(X)
    max_iter = 10000000
    rand_col = np.random.randint(0, F.shape[1])
    
    for i in range(rank_x):
        u = F[:, rand_col]
        for _ in range(max_iter):            
            w = np.dot(u.T, X)/np.dot(u.T, u)
            w = w/np.linalg.norm(w)
            t_ = t
            t = np.dot(E, w)/np.dot(w.T,w)
            q = np.dot(t.T, F)/np.dot(t.T,t)
            q_ = q/np.linalg.norm(q)
            u = np.dot(F,q.T)/np.dot(q.T,q)
            if(np.allclose(t_, t, rtol = 1e-6, atol = 1e-6)):
                break
        
        p = np.dot(t.T,E)/np.dot(t,t.T)
        p= p/np.linalg.norm(p)
        t = t*np.linalg.norm(p)
        w = w*np.linalg.norm(p)
        
        b = np.dot(u.T, u)/np.dot(t.T, t)
        W[i] = w
        B[i] = b
        P[i] = p
        Q[i] = q
        
        t_ = t.reshape([-1, 1])
        p_ = p.reshape([-1, 1])
        q_ = q.reshape([-1, 1])
        E = E - t_ @ p_.T
        F = F - b * t_ @ q_.T
        W[i] = w
        B[i] = b
        P[i] = p
        Q[i] = q
        
    

    return W,B,P,Q

def predict_pls(X, Y, X_test, r):
    W,B,P,Q = PLS(X, Y)
    E = X_test
    T_hat = np.zeros(X_test.shape[0])
    Y_pred = np.zeros((X_test.shape[0], Y.shape[1]))
    for i in range(r):
        T_hat = E @ W[i]
        E = E - T_hat[:, None] @ P[i][:, None].T
        inter = T_hat[:, None] @ Q[i][None, :]
        Y_pred += B[i] * inter
    return Y_pred

### Optimized using Cython

In [4]:
%%cython --annotate

import numpy as np
cimport numpy as np
from libc.stdio cimport printf
from cython.parallel import prange, parallel
import cython
from libc.math cimport sqrt


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef fast_dot(float[:, :] A, float[:, :] B):
    '''prepare data'''
    cdef int A_row = A.shape[0]
    cdef int A_col = A.shape[1]
    cdef int B_row = B.shape[0]
    cdef int B_col = B.shape[1]
    cdef float[:, :] mat_1 = A
    cdef float[:,:] mat_2 = B
    cdef float[:, :] result = np.zeros([A.shape[0],B.shape[1]],dtype=np.float32)
    '''begin dot''' 
    cdef int i=0, j=0, k=0
    for i in range(A_row):
        for j in xrange(B_col):
            result[i,j] = 0.0
            for k in xrange(A_col):
                result[i,j] += mat_1[i,k] * mat_2[k,j] 
#     print(np.multiply(result, 1))
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef fast_dot_n1(float[:, :] A, float[:, :] B):
    '''prepare data'''
    cdef int A_row = A.shape[0]
    cdef int A_col = A.shape[1]
    cdef int B_row = B.shape[0]
    cdef int B_col = 1
    cdef float[:, :] mat_1 = A
    cdef float[:,:] mat_2 = B
    cdef float[:] result = np.zeros(A.shape[0],dtype=np.float32)
    '''begin dot''' 
    cdef int i=0, j=0, k=0
    for i in range(A_row):
        for j in xrange(1):
            result[i] = 0.0
            for k in xrange(A_col):
                result[i] += mat_1[i,k] * mat_2[k,0] 
#     print(np.multiply(result, 1))
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef fast_dot_11(float[:, :] A, float[:, :] B):
    '''prepare data'''
    cdef int A_col = A.shape[1]
    cdef int B_row = B.shape[0]
    cdef float[:,:] mat_1 = A
    cdef float[:,:] mat_2 = B
    cdef float result = 0.0
    '''begin dot''' 
    cdef int i=0, j=0, k=0
    for i in range(A_col):
        result += mat_1[0,i] * mat_2[i,0] 
#     print(np.multiply(result, 1))
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef scalar_multiply(float a, float[:, :] b):
    cdef float[:, :] mat = b
    cdef int blen = b.shape[0]
    cdef int bwid = b.shape[1]
    cdef int i,j
    for i in range(blen):
        for j in range(bwid):
            mat[i, j] = a*b[i, j]
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef scalar_division(float[:] vec, float sca):
    cdef float[:] mat = vec
    cdef int blen = vec.shape[0]
    cdef int i
    for i in range(blen):
        mat[i] = vec[i]/sca
    return mat


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef scalar_division_1d2d(float[:,:] vec, float sca):
#     ans = np.zeros([vec.shape[0], 1]).astype(np.float32)
    cdef float[:, :] mat = np.zeros([vec.shape[0], 1],dtype=np.float32)
    cdef int blen = vec.shape[0]
    cdef int i
    for i in range(blen):
        mat[i,0] = vec[i,0]/sca
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef minus_2d(float[:,:] A, float[:,:] B):
    cdef float[:, :] mat = np.zeros([A.shape[0], A.shape[1]],dtype=np.float32)
    cdef int i,j
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            mat[i,j] = A[i,j] - B[i,j]
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef dot_1d(float[:,:] v1, float[:,:] v2):
    cdef float result = 0.0
    cdef int i = 0
    cdef int length = v1.shape[0]
    cdef double el1 = 0
    cdef double el2 = 0
    for i in range(length):
        el1 = v1[i,0]
        el2 = v2[0,i]
        result += el1*el2
    return result


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef float norm_1d(float[:,:] v1):
    cdef float result = 0.0
    cdef int i = 0
    cdef int length = v1.shape[0]
    for i in range(length):
        result += v1[i,0]*v1[i,0]
    result = sqrt(result)
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef PLS_cython(float[:,:] X, float[:,:] Y,int r,int max_iter,int rand_col):
#     X = X.astype(np.float32)
#     Y = Y.astype(np.float32)
    cdef float[:, :] E_h = X
#     cdef float[:, :] F_h = Y.reshape([Y.shape[0], 1])
    cdef float[:, :] F_h = Y
    W = {}
    B = {}
    P = {}
    Q = {}
#     cdef float[:,:,:] W = np.empty([r,X.shape[1],1],dtype=np.float32)
#     cdef float[:,:] B = np.empty([r,1],dtype=np.float32)
#     cdef float[:,:,:] P = np.empty([r,X.shape[1], 1],dtype=np.float32)
#     cdef float[:,:,:] Q = np.empty([r,Y.shape[1], 1],dtype=np.float32)
#     cdef unordered_map[int, int] mapa
#     cdef pair[int, int] entry_w
    
    
    cdef float[:,:] u = np.zeros([Y.shape[0],1],dtype=np.float32)
    cdef float[:,:] w = np.zeros([X.shape[1],1],dtype=np.float32)
    cdef float[:, :] t = np.zeros([X.shape[0], 1],dtype=np.float32)
    cdef float[:, :] t_ = np.zeros([X.shape[0], 1],dtype=np.float32)
    cdef float[:, :] p = np.zeros([X.shape[1], 1],dtype=np.float32)
    cdef float[:, :] q = np.zeros([Y.shape[1], 1],dtype=np.float32)
    cdef float[:, :] q_ = np.zeros([Y.shape[1], 1],dtype=np.float32)
    cdef float b = 0.0
    cdef int i,j,k
    cdef int yshape
    cdef float pnorm

    for i in range(r):
#         if i == 0:
#             u = Y.reshape(-1,1)
#         else:
#             u = Y
        yshape = Y.shape[0]
        for k in range(yshape):
            u[k,0] = Y[k,rand_col]
        for j in range(max_iter):
            w = np.dot(X.T, u)/ dot_1d(u.T ,u)
            #step 3
            w = scalar_division_1d2d(w,norm_1d(w))

            t_ = t
            #step 4

            t = np.dot(X, w)/dot_1d(w.T, w)
            q = np.dot(Y.T, t)/dot_1d(t.T, t)
            q_ = scalar_division_1d2d(q,norm_1d(q))
            u = np.dot(Y, q)/dot_1d(q.T, q)
            if np.allclose(t_, t, rtol = 1e-6, atol = 1e-6):
                break





        #step5-8 omitted
        #step 9

        p = np.dot(X.T, t)/dot_1d(t.T, t)
        #step 10
        pnorm = norm_1d(p)
        p = scalar_division_1d2d(p, pnorm)       


        #step 11
        t = scalar_multiply(pnorm,t)
        #step 12
        w = scalar_multiply(pnorm,w)
        #step 13
        b = np.dot(u.T, t)/dot_1d(t.T, t)
#         print(b.shape)
        # Calculation of the residuals
        W[i] = w
        B[i] = b
        P[i] = p
        Q[i] = q
        
        E_h = minus_2d(E_h,np.dot(t,p.T))
        F_h = minus_2d(F_h,np.dot(scalar_multiply(b,t),q.T))
#         print(F_h.shape)
        #Replace X and Y
        X = E_h
        Y = F_h
        #update W and B
#         entry_w.first = i
#         entry_w.second = w
    return W,B,P,Q

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef predict_cython(float[:,:] X, float[:,:] Y, float[:,:] X_test,int r):
    W,B,P,Q = PLS_cython(X, Y, r, 10000000, np.random.randint(0, Y.shape[1]))
    cdef float[:,:] E = X   
    cdef float[:,:] T_hat = np.zeros([X_test.shape[0], 1]).astype(np.float32)
    cdef float[:,:] Y_pred = np.zeros([X_test.shape[0], Y.shape[1]]).astype(np.float32)
    cdef int i,j,k
    for i in range(r):
        T_hat = np.dot(E,W[i])
        E = E - np.dot(T_hat, P[i].T)
        inter = np.dot(T_hat ,Q[i].T)
        for k in range(Y_pred.shape[0]):
            for j in range(Y_pred.shape[1]):
                Y_pred[k,j] += B[i] * inter[k,j]
    return Y_pred

In [5]:
# import pandas as pd
# np.random.seed(9856)
# x1 = np.random.normal(1, .2, 100
#                      )
# x2 = np.random.normal(5, .4, 100)
# x3 = np.random.normal(12, .8, 100)

# def generate_sim(x1, x2, x3):

#     sim_data = {'x1' : x1,
#                 'x2' : x2, 
#                 'x3' : x3,
#                 'x4' : 5*x1,
#                 'x5' : 2*x2,
#                 'x6' : 4*x3,
#                 'x7' : 6*x1,
#                 'x8' : 5*x2,
#                 'x9' : 4*x3,
#                 'x10' : 2*x1,
#                 'y0' : 3*x2 + 3*x3,
#                 'y1' : 6*x1 + 3*x3,
#                 'y2' : 7*x2 + 2*x1}

#     # convert data to csv file
#     data = pd.DataFrame(sim_data)

#     sim_predictors = data.drop(['y0', 'y1', 'y2'], axis = 1).columns.tolist()
#     sim_values = ['y0', 'y1', 'y2']
# #     sim_values = ['y0']

#     pred = data[sim_predictors].values
#     val = data[sim_values].values
    
#     return pred, val

# pred, val = generate_sim(x1,x2,x3)
# X_test = pred.astype(np.float32)
# y_test = val.astype(np.float32)

# test_x1 = np.random.normal(1, .2, 3000).astype(np.float32)
# test_x2 = np.random.normal(5, .4, 3000).astype(np.float32)
# test_x3 = np.random.normal(12, .8, 3000).astype(np.float32)

# pred_test, pred_val = generate_sim(test_x1, test_x2, test_x3)
# X_train = pred_test.astype(np.float32)
# y_train = pred_val.astype(np.float32)

In [6]:
# r = np.linalg.matrix_rank(X_train)
# rand_col = np.random.randint(0, y_train.shape[1])
# max_iter = 10000000

In [7]:
# %timeit PLS_cython(X_train, y_train,r,max_iter,rand_col)

In [8]:
# def PLS(X, Y):
#     t_ = 0
#     t = 0
#     W = {}
#     B = {}
#     P = {}
#     Q = {}
#     E = X
#     F = Y
    
#     y_shape = Y.shape[1]
#     rank_x = np.linalg.matrix_rank(X)
#     max_iter = 10000000
#     rand_col = np.random.randint(0, F.shape[1])
# #     u = np.zeros(Y.shape[0],dtype=np.float32)
# #     w = np.zeros([X.shape[1],1],dtype=np.float32)
# #     t = np.zeros([X.shape[0], 1],dtype=np.float32)
# #     t_ = np.zeros([X.shape[0], 1],dtype=np.float32)
# #     p = np.zeros([X.shape[1], 1],dtype=np.float32)
# #     q = np.zeros([Y.shape[1], 1],dtype=np.float32)
# #     q_ = np.zeros([Y.shape[1], 1],dtype=np.float32)
# #     b = 0.0
    
#     for i in range(rank_x):
# #         for j in range(Y.shape[0]):
# #             u[j] = Y[j,rand_col]
#         u = Y[:,rand_col]
#         for _ in range(max_iter):            
#             w = np.dot(u.T, X)/np.dot(u.T, u)
#             w = w/np.linalg.norm(w)
#             t_ = t
#             t = np.dot(E, w)/np.dot(w.T,w)
#             q = np.dot(t.T, F)/np.dot(t.T,t)
#             q_ = q/np.linalg.norm(q)
#             u = np.dot(F,q.T)/np.dot(q.T,q)
#             if(np.allclose(t_, t, rtol = 1e-6, atol = 1e-6)):
#                 break
        
#         p = np.dot(t.T,E)/np.dot(t,t.T)
#         p= p/np.linalg.norm(p)
#         t = t*np.linalg.norm(p)
#         w = w*np.linalg.norm(p)
        
#         b = np.dot(u.T, u)/np.dot(t.T, t)
#         W[i] = w
#         B[i] = b
#         P[i] = p
#         Q[i] = q
        
#         t_ = t.reshape([-1, 1])
#         p_ = p.reshape([-1, 1])
#         q_ = q.reshape([-1, 1])
#         E = E - t_ @ p_.T
#         F = F - b * t_ @ q_.T
#         W[i] = w
#         B[i] = b
#         P[i] = p
#         Q[i] = q
        
    

#     return W,B,P,Q

In [9]:
# %timeit PLS(X_train, y_train)

In [10]:

# %timeit PLS_cython_opt(X_train.astype("double"), y_train.astype("double"))

### Optimized using Cython with ndarray

In [11]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.stdio cimport printf
from cython.parallel import prange, parallel
import cython
cimport cython
from libc.math cimport sqrt

ctypedef np.double_t DTYPE_t
ctypedef np.int64_t TTYPE_t

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef DTYPE_t dot_1d(np.ndarray[DTYPE_t,ndim = 2] v1, np.ndarray[DTYPE_t,ndim = 2] v2):
    cdef DTYPE_t result = 0
    cdef int i = 0
    cdef int length = v1.shape[0]
    cdef double el1 = 0
    cdef double el2 = 0
    for i in range(length):
        el1 = v1[i,0]
        el2 = v2[0,i]
        result += el1*el2
    return result

cpdef np.ndarray[DTYPE_t, ndim=2] scalar_multiply(DTYPE_t a, np.ndarray[DTYPE_t, ndim=2] b):
    cdef np.ndarray[DTYPE_t, ndim=2] mat = b.copy()
    cdef TTYPE_t blen = b.shape[0]
    cdef TTYPE_t bwid = b.shape[1]
    for i in range(blen):
        for j in range(bwid):
            mat[i, j] = a*b[i, j]
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef np.ndarray[DTYPE_t, ndim=1] scalar_division(np.ndarray[DTYPE_t, ndim=1] vec, DTYPE_t sca):
    cdef np.ndarray[DTYPE_t, ndim=1] mat = vec.copy()
    cdef TTYPE_t blen = vec.shape[0]
    cdef int i
    with cython.nogil, parallel():
        for i in prange(blen):
            mat[i] = vec[i]/sca
    return mat


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef np.ndarray[DTYPE_t, ndim=2] scalar_division_1d2d(np.ndarray[DTYPE_t, ndim=2] vec, DTYPE_t sca):
    cdef np.ndarray[DTYPE_t, ndim=2] mat = vec.copy()
    cdef TTYPE_t blen = vec.shape[0]
    cdef TTYPE_t i
    with cython.nogil, parallel():
        for i in prange(blen):
            mat[i,0] = vec[i, 0]/sca
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef np.ndarray[DTYPE_t, ndim=2] minus_2d(np.ndarray[DTYPE_t, ndim=2] A, np.ndarray[DTYPE_t, ndim=2] B):
    cdef np.ndarray[DTYPE_t, ndim=2] mat = A.copy()
    cdef int i, j
    with cython.nogil, parallel():
        for i in prange(A.shape[0]):
            for j in prange(A.shape[1]):
                mat[i,j] = A[i,j] - B[i,j]
    return mat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef double norm_1d(double[:,:] v1):
    cdef double result = 0
    cdef int i = 0
    cdef int length = v1.shape[0]
    for i in range(length):
        result += v1[i,0]*v1[i,0]
    result = sqrt(result)
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cpdef PLS_cython_ndarray(np.ndarray[DTYPE_t, ndim=2] X, np.ndarray[DTYPE_t, ndim=2] Y,r,max_iter,rand_col):
    cdef np.ndarray[DTYPE_t, ndim=2] E_h = X.copy()
    cdef np.ndarray[DTYPE_t, ndim=2] F_h = Y.copy()
#     cdef int r = np.linalg.matrix_rank(X)
#     cdef int rand_col = np.random.randint(0, X.shape[1])
#     cdef int max_iter = 10000000
    W = {}
    B = {}
    P = {}
    Q = {} 
    cdef np.ndarray[DTYPE_t, ndim=2] u = np.zeros([Y.shape[0],1])
    cdef np.ndarray[DTYPE_t, ndim=2] w = np.zeros([X.shape[1],1])
    cdef np.ndarray[DTYPE_t, ndim=2] t = np.zeros([X.shape[0], 1])
    cdef np.ndarray[DTYPE_t, ndim=2] t_ = np.zeros([X.shape[0], 1])
    cdef np.ndarray[DTYPE_t, ndim=2] p = np.zeros([X.shape[1], 1])
    cdef np.ndarray[DTYPE_t, ndim=2] q = np.zeros([Y.shape[1], 1])
    cdef np.ndarray[DTYPE_t, ndim=2] q_ = np.zeros([Y.shape[1], 1])
    cdef DTYPE_t b = 0.0
    cdef int i,j,k
    cdef int yshape
    cdef DTYPE_t pnorm
    
    for i in range(r):
#         if i == 0:
#             u = Y.reshape(-1,1)
#         else:
#             u = Y
        yshape = Y.shape[0]
        for k in range(yshape):
            u[k,0] = Y[k,rand_col]
        for j in range(max_iter):
            w = np.dot(X.T, u)/ dot_1d(u,u.T)
            
            #step 3
            w = w/norm_1d(w)

            t_ = t
            #step 4
            t = np.dot(X, w)/dot_1d(w,w.T)
            
            q = np.dot(Y.T, t)/dot_1d(t,t.T)
            
            q_ = q/norm_1d(q)
            u = np.dot(Y, q)/dot_1d(q,q.T)
            if np.allclose(t_, t, rtol = 1e-6, atol = 1e-6):
                break





        #step5-8 omitted
        #step 9
        
        p = np.dot(X.T, t)/dot_1d(t.T, t)
        
        #step 10
        pnorm = norm_1d(p) 
        p = p/pnorm      
        

        #step 11
        t = t* pnorm
        #step 12
        w = w *pnorm
        #step 13
        b = np.dot(u.T, t)/dot_1d(t.T, t)
#         print(b.shape)
        # Calculation of the residuals
        W[i] = w
        B[i] = b
        P[i] = p
        Q[i] = q
        
        E_h = minus_2d(E_h,np.dot(t,p.T))
        F_h = minus_2d(F_h,np.dot(scalar_multiply(b,t),q.T))
#         print(F_h.shape)
        #Replace X and Y
        X = E_h
        Y = F_h
        #update W and B
#         entry_w.first = i
#         entry_w.second = w
    return W,B,P,Q

# cpdef predict_cython(np.ndarray[DTYPE_t, ndim=2] X, np.ndarray[DTYPE_t, ndim=2] Y,np.ndarray[DTYPE_t, ndim=2] X_test,int r):
#     W,B,P,Q = PLS_cython(X,Y)
#     cdef np.ndarray[DTYPE_t, ndim=2] E_h = X_test.copy()
#     cdef np.ndarray[DTYPE_t, ndim=2] y_pred = np.zeros((X_test.shape[0],1))
#     cdef np.ndarray[DTYPE_t, ndim=2] t_hat = np.zeros((X_test.shape[0],1))
#     cdef int i,j
#     for i in range(r):
#         t_hat = np.dot(E_h, W[i])
#         E_h = E_h - np.dot(t_hat, P[i].T)
#         for j in range(y_pred.shape[0]):
#             y_pred[j,0] = y_pred[j,0] + B[i] * t_hat[j,0]
#     return y_pred[:,0]
cpdef predict_cython_nd(np.ndarray[DTYPE_t, ndim=2] X, np.ndarray[DTYPE_t, ndim=2] Y, np.ndarray[DTYPE_t, ndim=2] X_test,int r):
    W,B,P,Q  = PLS_cython_ndarray(X, Y, r, 10000000, np.random.randint(0, Y.shape[1]))
    cdef np.ndarray[DTYPE_t, ndim=2] E = X   
    cdef np.ndarray[DTYPE_t, ndim=2] T_hat = np.zeros([X_test.shape[0], 1])
    cdef np.ndarray[DTYPE_t, ndim=2] Y_pred = np.zeros([X_test.shape[0], Y.shape[1]])
    cdef int i,j,k
    for i in range(r):
        T_hat = np.dot(E,W[i])
        E = E - T_hat @ P[i].T
        inter = T_hat @ Q[i].T
        for k in range(Y_pred.shape[0]):
            for j in range(Y_pred.shape[1]):
                Y_pred[k,j] += B[i] * inter[k,j]
    return Y_pred

### Simulate dataset

In [12]:
np.random.seed(9856)
x1 = np.random.normal(1, .2, 100
                     )
x2 = np.random.normal(5, .4, 100)
x3 = np.random.normal(12, .8, 100)

def generate_sim(x1, x2, x3):

    sim_data = {'x1' : x1,
                'x2' : x2, 
                'x3' : x3,
                'x4' : 5*x1,
                'x5' : 2*x2,
                'x6' : 4*x3,
                'x7' : 6*x1,
                'x8' : 5*x2,
                'x9' : 4*x3,
                'x10' : 2*x1,
                'y0' : 3*x2 + 3*x3,
                'y1' : 6*x1 + 3*x3,
                'y2' : 7*x2 + 2*x1}

    # convert data to csv file
    data = pd.DataFrame(sim_data)

    sim_predictors = data.drop(['y0', 'y1', 'y2'], axis = 1).columns.tolist()
    sim_values = ['y0', 'y1', 'y2']
#     sim_values = ['y0']

    pred = data[sim_predictors].values
    val = data[sim_values].values
    
    return pred, val

X_test_sim, y_test_sim = generate_sim(x1,x2,x3)
# X_test = pred.astype(np.float32)
# y_test = val.astype(np.float32)

test_x1 = np.random.normal(1, .2, 5000)
test_x2 = np.random.normal(5, .4, 5000)
test_x3 = np.random.normal(12, .8, 5000)

X_train_sim, y_train_sim= generate_sim(test_x1, test_x2, test_x3)
# X_train = pred_test.astype(np.float32)
# y_train = pred_val.astype(np.float32)

In [13]:
r = np.linalg.matrix_rank(X_train_sim)
rand_col = np.random.randint(0, y_train_sim.shape[1])
max_iter = 10000000

In [14]:
%timeit PLS_cython_ndarray(X_train_sim, y_train_sim,r,max_iter,rand_col)

4.06 ms ± 83.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
y_train_sim.shape

(5000, 3)

In [16]:
%timeit predict_cython_nd(X_train_sim, y_train_sim, X_test_sim, r)

5 ms ± 99.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit PLS_cython(X_train_sim.astype(np.float32), y_train_sim.astype(np.float32),r,max_iter,rand_col)

2.08 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit PLS(X_train_sim, y_train_sim)

3.87 ms ± 85.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
 %timeit predict_cython(X_train_sim.astype(np.float32), y_train_sim.astype(np.float32), X_test_sim.astype(np.float32), r)

In [ ]:
%timeit predict_pls(X_train_sim,y_train_sim, X_test_sim, r)

### JIT optimization

In [21]:
import numba
from numba import jit, vectorize, float64, int64

In [22]:
def nipals(X):
    for i in range(X.shape[1]):
        t_h = X[:, i]
        p_h = (t_h.T@X)/(t_h.T@t_h)
        p_h = p_h/np.norm(p_h)
        t_new = X@p_h/(p_h.T@p_h)
        if(np.allclose(t_h, t_new)):
            return t_h, p_h

In [23]:
@jit
def update1(X,u,t,t_,E,F):
    w = np.dot(u.T, X)/np.dot(u.T, u)
    w = w/np.linalg.norm(w)
    t_ = t
    t = np.dot(E, w)/np.dot(w.T,w)
    q = np.dot(t.T, F)/np.dot(t.T,t)
    q_ = q/np.linalg.norm(q)
    u = np.dot(F,q.T)/np.dot(q.T,q)
    return w,t,t_,q,u

In [24]:
@jit
def update2(t,w,u,q,i,E,F):
    p = np.dot(t.T,E)/np.dot(t,t.T)
    p= p/np.linalg.norm(p)
    t = t*np.linalg.norm(p)
    w = w*np.linalg.norm(p)
        
    b = np.dot(u.T, u)/np.dot(t.T, t)
#     W[i] = w
#     B[i] = b
#     P[i] = p
#     Q[i] = q
        
    t_ = t.reshape([-1, 1])
    p_ = p.reshape([-1, 1])
    q_ = q.reshape([-1, 1])
    E = E - t_ @ p_.T
    F = F - b * t_ @ q_.T
    return E,F,w,b,p

In [65]:
def PLS_optimized_jit(X, Y):
    t_ = 0
    t = 0
    W = {}
    B = {}
    P = {}
    Q = {}
    E = X
    F = Y
    
    y_shape = Y.shape[1]
    rank_x = np.linalg.matrix_rank(X)
    max_iter = 10000000
    rand_col = np.random.randint(0, F.shape[1])
    
    for i in range(rank_x):
        u = F[:, rand_col]
        for _ in range(max_iter):            
#             w = np.dot(u.T, X)/np.dot(u.T, u)
#             w = w/np.linalg.norm(w)
#             t_ = t
#             t = np.dot(E, w)/np.dot(w.T,w)
#             q = np.dot(t.T, F)/np.dot(t.T,t)
#             q_ = q/np.linalg.norm(q)
#             u = np.dot(F,q.T)/np.dot(q.T,q)
            w,t,t_,q,u = update1(X,u,t,t_,E,F)
            if(np.allclose(t_, t, rtol = 1e-6, atol = 1e-6)):
                break
        
#         p = np.dot(t.T,E)/np.dot(t,t.T)
#         p= p/np.linalg.norm(p)
#         t = t*np.linalg.norm(p)
#         w = w*np.linalg.norm(p)
        
#         b = np.dot(u.T, u)/np.dot(t.T, t)
#         W[i] = w
#         B[i] = b
#         P[i] = p
#         Q[i] = q
        
#         t_ = t.reshape([-1, 1])
#         p_ = p.reshape([-1, 1])
#         q_ = q.reshape([-1, 1])
#         E = E - t_ @ p_.T
#         F = F - b * t_ @ q_.T
        E,F,w,b,p = update2(t,w,u,q,i,E,F)
        W[i] = w
        B[i] = b
        P[i] = p
        Q[i] = q
    return W,B,P,Q

def predict_pls_jit(X, Y, X_test, r):
    W,B,P,Q = PLS_optimized_jit(X, Y)
    E = X_test
    T_hat = np.zeros(X_test.shape[0])
    Y_pred = np.zeros((X_test.shape[0], Y.shape[1]))
    for i in range(r):
        T_hat = E @ W[i]
        E = E - T_hat[:, None] @ P[i][:, None].T
        inter = T_hat[:, None] @ Q[i][None, :]
        Y_pred += B[i] * inter
    return Y_pred

In [66]:
%timeit PLS_optimized_jit(X_train_sim, y_train_sim)

2.87 ms ± 72.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### real dataset

In [94]:
import sklearn.datasets
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
b = sklearn.datasets.load_boston()

In [95]:
X = pd.DataFrame(b.data, columns=b.feature_names)

In [96]:
Y = pd.concat([
    pd.Series(b.target, name='MEDV'),
    X[['RM', 'TAX']],
], axis=1) 

X = X.drop(['RM', 'TAX'], axis=1)

In [97]:
X.shape

(506, 11)

In [98]:
Y.shape

(506, 3)

In [99]:
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1)
# X_train = X_train.values
# X_test = X_test.values
# Y_train = Y_train.values
# Y_test = Y_test.values

In [100]:
# r = np.linalg.matrix_rank(X_train)
# rand_col = np.random.randint(0, Y_train.shape[1])
# max_iter = 10000000

In [102]:
# %timeit PLS_cython_ndarray(X_train, y_train,r,max_iter,rand_col)

In [76]:
# %timeit PLS_cython(X_train.astype(np.float32), Y_train.astype(np.float32),r,max_iter,rand_col)

In [103]:
# %timeit PLS(X_train, Y_train)

In [104]:
# %timeit PLS_optimized_jit(X_train, Y_train)

In [ ]:
y_pred_PLS = predict_pls(X_train,y_train, X_test, r)

In [ ]:
y_pred_nd = predict_cython_nd(X_train, Y_train, X_test, r)

In [ ]:
y_pred_cython = predict_cython(X_train.astype(np.float32), y_train.astype(np.float32), X_test_sim.astype(np.float32), r)

In [ ]:
y_pred_jit = predict_pls_jit(X_train,y_train, X_test, r)

In [106]:
wine_data = pd.read_csv("winequality-red.csv")
wine_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [107]:
wine_d = wine_data.drop(["quality","residual sugar"], axis = 1)
wine_l = wine_data[["quality","residual sugar"]]

In [108]:
X_train, X_test, y_train, y_test = train_test_split(wine_d, wine_l, test_size=0.1)
X_train = X_train + 0.00001
y_train = y_train + 0.00001
X_test = X_test + 0.00001
y_test = y_test + 0.00001
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [118]:
y_train.shape


(1439, 2)

In [119]:
X_train.shape

(1439, 10)

In [120]:
y_test.shape

(160, 2)

In [110]:
r = np.linalg.matrix_rank(X_train)
rand_col = np.random.randint(0, y_train.shape[1])
max_iter = 100000

In [111]:
%timeit PLS_cython_ndarray(X_train, y_train,r,max_iter,rand_col)

5.03 ms ± 262 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [112]:
# %timeit PLS_cython(X_train.astype(np.float32), y_train.astype(np.float32),r,max_iter,rand_col)

In [113]:
%timeit PLS(X_train, y_train)

7.55 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [114]:
%timeit PLS_optimized_jit(X_train, y_train)

5.03 ms ± 307 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [128]:
y_pred_PLS = predict_pls(X_train,y_train, X_test, r)

In [129]:
y_pred_nd = predict_cython_nd(X_train, y_train, X_test, r)

In [130]:
# y_pred_cython = predict_cython(X_train.astype(np.float32), y_train.astype(np.float32), X_test_sim.astype(np.float32), r)

In [131]:
y_pred_jit = predict_pls_jit(X_train,y_train, X_test, r)

In [132]:
from sklearn.metrics import mean_squared_error

In [133]:
mean_squared_error(y_test, y_pred_PLS)

14.666505650388778

In [134]:
mean_squared_error(y_test, y_pred_nd)

1.5083407834770202e+54

In [135]:
mean_squared_error(y_test, y_pred_jit)

14.666508525388604